# This is a notebook to demonstrate k-space to image space conversion with MRI


Also add MR simulation of frequency and phase encoding? 

https://thepythoncodingbook.com/2021/08/30/2d-fourier-transform-in-python-and-fourier-synthesis-of-images/

# be able to..
1) transform between k space and not k-space  $\checkmark$

2) select region of kspace to transform and see resulting image $\checkmark$

2) watch as image is constructed from added points in kspace (like CT reconstruction) with the plane waves overlaid (and path/points are shown in k space as it's added on $\checkmark$

4) observe the path (epi or radial etc) and how image is formed $\checkmark$

5) How only a little more than half of ks-space is needed

5) phase and frequency encoding? or is that too advanced?

root = Tk()
frm = ttk.Frame(root, padding=10)
frm.grid()
ttk.Label(frm, text="Hello World!").grid(column=0, row=0)
ttk.Button(frm, text="Quit", command=root.destroy).grid(column=1, row=0)
root.mainloop()

In [42]:
import numpy as np
import matplotlib
import matplotlib.pyplot as pl
import csv
import scipy.optimize as op
import scipy.stats
from scipy.optimize import curve_fit
import scipy.io
from scipy.stats import rice

import random
import pickle
import seaborn as sns

import numpy as np
from scipy.stats import ttest_ind, ttest_ind_from_stats, wilcoxon,ttest_rel, pearsonr
from scipy.special import stdtr
import csv
import pandas as pd

import sys 
import os
#sys.path.append(os.path.abspath("/Users/neuroimaging/Desktop/MR-code/Python_Code"))

import matplotlib.image as mpimg


from tkinter import *
from tkinter import ttk

from mpl_toolkits.mplot3d import Axes3D

import pydicom as dicom


In [43]:
%matplotlib

Using matplotlib backend: MacOSX


# Show Image

$FOV_{img} = 1361 x 1361$ \
$\Delta x = 1$ 

In [54]:
#img=mpimg.imread('doggy.jpeg')
ds = dicom.dcmread('Axial.dcm')
img = ds.pixel_array
#img = #img[530:-530,:-1,0]
print(np.shape(img))

#img = img[1:,33:-33]
pl.imshow(img,cmap = 'gray')
pl.show()

fov_img = 256

(256, 256)


# Get k-space through FFT of Image
$FOV_{kspace} = \frac{1}{\Delta x} = 1$\
$\Delta k_x = \frac{1}{FOV_{img}} = \frac{1}{1361} $ 

In [55]:
ft = np.fft.ifftshift(img)
ft = np.fft.fft2(ft)
kspace = np.fft.fftshift(ft) #moves zero frequency component to center
kx_ax = np.fft.fftfreq(np.shape(img)[0],1) #goes from -.5 to +.5 frequency
print('min kx:',min(kx_ax),'\nmax kx:', max(kx_ax),'\nDelta kx = ',1/fov_img)
pl.imshow(np.abs(kspace),vmin = 0,vmax = 4000,extent = [-.5, .5, -.5, .5],cmap = 'gray')
pl.colorbar()
pl.show()

min kx: -0.5 
max kx: 0.49609375 
Delta kx =  0.00390625


# Get image back with iFFT of k-space

In [56]:
ift = np.fft.fftshift(kspace)
ift = np.fft.ifft2(ift)
iimg = np.fft.ifftshift(ift)
pl.imshow(np.abs(iimg),cmap = 'gray')
pl.show()


# Now get ROI and crop k-space
give it a center, and x and y dimensions. will make a rectangle

In [57]:
def DrawRect(kspace,fov_img,center,xdim,ydim): #takes it in idx, not frequency values
    kspace_cropped = kspace
    minx = center[0] - xdim
    maxx = center[0] + xdim
    miny = center[1] - ydim
    maxy = center[1] + ydim
    for j in range(len(kspace)):
        for k in range(len(kspace)):
            if j < minx or j > maxx or k < miny or k > maxy : 
                #print(kspace_cropped[k,j])
                kspace_cropped[k,j] = 0 #k is y, j is x. 
                #print(kspace_cropped[k,j])
            #else:
                #print('not')
    return kspace_cropped

In [58]:
fig, (ax1, ax2) = pl.subplots(1,2)

rect_kspace = DrawRect(kspace,fov_img,[round(fov_img/2),round(fov_img/2)],50,50)
ax1.imshow(np.abs(rect_kspace),vmin = 0,vmax = 4000,extent = [-.5, .5, -.5, .5],cmap = 'gray')

ift = np.fft.fftshift(rect_kspace)
ift = np.fft.ifft2(ift)
cropped_iimg = np.fft.ifftshift(ift)
ax2.imshow(np.abs(cropped_iimg),cmap = 'gray')
pl.show()


# Now watch live reconstruction of the image from k-space

# do as spiral.... 


In [59]:
# assumed odd and equal nummber of nx and ny
# makes list of indices to do a spiral
# see 8/20/22 notes
def Spiral(nx):
    SpiralArray_x = [] 
    SpiralArray_y = []
    center = int(nx/2)
    # add center as first 
    x,y = center,center
    SpiralArray_x.append(x)
    SpiralArray_y.append(y)
    for j in np.arange(1,nx,2): #from 0 to the number of nx there is in steps of 2
        # do x = x+1 and then y = y-1 a certain number of times
        for k in range(j):
            x = x+1
            SpiralArray_x.append(x)
            SpiralArray_y.append(y)
        for m in range(j):
            y = y-1
            SpiralArray_x.append(x)
            SpiralArray_y.append(y)
        for p in range(j+1):
            x = x-1
            SpiralArray_x.append(x)
            SpiralArray_y.append(y)
        for q in range(j+1):
            y = y+1
            SpiralArray_x.append(x)
            SpiralArray_y.append(y)
    for v in range(nx-1):
        x = x+1
        SpiralArray_x.append(x)
        SpiralArray_y.append(y)
        
    SpiralArray = np.vstack((SpiralArray_x,SpiralArray_y))
    
    return SpiralArray

#this will avoid doubling with mirroring (see 8/29/22)
def cropped_spiral(nx):
    SpiralArray_x = [] 
    SpiralArray_y = []
    center = int(nx/2)
    # add center as first 
    x,y = center,center
    SpiralArray_x.append(x)
    SpiralArray_y.append(y)
    for j in range(int(nx/2)):
        y = center + j
        x = center
        SpiralArray_x.append(x)
        SpiralArray_y.append(y)
        for k in range(j):
            x = x+1 
            SpiralArray_x.append(x)
            SpiralArray_y.append(y)
        for m in range(j*2):
            y = y - 1
            SpiralArray_x.append(x)
            SpiralArray_y.append(y)
        for p in range(j-1):
            x = x-1
            SpiralArray_x.append(x)
            SpiralArray_y.append(y)
    
    SpiralArray = np.vstack((SpiralArray_x,SpiralArray_y))
    
    return SpiralArray


In [60]:
cropped_kspace = rect_kspace[round(fov_img/2)-50:round(fov_img/2)+51,round(fov_img/2)-50:round(fov_img/2)+51]

#fig, (ax1,ax2,ax3) = pl.subplots(1,3)
SpiralAcquisition = cropped_spiral(len(cropped_kspace)) #get spiral acuisition points
center = int(len(cropped_kspace)/2)

spiraling_k = np.zeros(np.shape(cropped_kspace))
superimpose_img = np.zeros(np.shape(cropped_kspace))

#ax3.imshow(img,cmap='gray')
#ax3.set_title('Final image')

stacked_iffts = np.zeros((len(cropped_kspace[0]),len(cropped_kspace[1]),int(len(SpiralAcquisition[0])/10)+1))
stacked_kspace = np.zeros(stacked_iffts)
ten_counter = np.arange(0,len(SpiralAcquisition[0]),10)
counter = 0
for j in np.arange(0,len(SpiralAcquisition[0])): #for each point in this spiral, i.e. all of k-space
    # get mask that gets that point and it's corresponding buddy 
    mask = np.zeros(np.shape(cropped_kspace))
    
    kx = SpiralAcquisition[0][j]
    ky = SpiralAcquisition[1][j]
    mask[kx,ky]  = 1
    # now get it reflected across the origin as well 
    dkx = center - kx
    dky = center - ky
    
    mirror_kx = center + dkx 
    mirror_ky = center + dky
    mask[mirror_kx, mirror_ky] = 1
        
    # now get those points in k-space
    kspace_point = cropped_kspace*mask 
    
    #show k-space growing
    spiraling_k = spiraling_k + kspace_point # and add it to show the growing k-space
    #ax1.imshow(np.abs(spiraling_k),vmin = 0,vmax = 6000,extent = [-.5, .5, -.5, .5],cmap = 'gray')
    #ax1.set_title('k-space used so far')
    
    
    # now get inverse ft of that point only every 10
    if j in ten_counter:
        stacked_kspace[:,:,counter] = np.abs(spiraling_k)
        ift = np.fft.fftshift(spiraling_k)
        ift = np.fft.ifft2(ift)
        iimg = np.fft.ifftshift(ift) # want to save this! 
        stacked_iffts[:,:,counter] = np.abs(iimg)
        counter = counter + 1
    
    # now begin to sum up these individual contributions
    #superimpose_img = superimpose_img + np.real(iimg)
    #ax2.imshow(np.abs(iimg),cmap = 'gray')
    #ax2.set_title('Image being produced')
    
    
    #pl.pause(.001)
    
    
    
    
    
with open('superposition.pickle','wb') as handle: 
    pickle.dump(stacked_iffts,handle)
with open('superposition_kspace.pickle','wb') as handle: 
    pickle.dump(stacked_kspace,handle)

    
    

In [61]:
cropped_kspace = rect_kspace[round(fov_img/2)-50:round(fov_img/2)+51,round(fov_img/2)-50:round(fov_img/2)+51]

#fig, (ax1,ax2,ax3) = pl.subplots(1,3)
SpiralAcquisition = cropped_spiral(len(cropped_kspace)) #get spiral acuisition points
center = int(len(cropped_kspace)/2)

spiraling_k = np.zeros(np.shape(cropped_kspace))
superimpose_img = np.zeros(np.shape(cropped_kspace))

#ax3.imshow(img,cmap='gray')
#ax3.set_title('Final image')

stacked_maskiffts = np.zeros((len(cropped_kspace[0]),len(cropped_kspace[1]),int(len(SpiralAcquisition[0])/10)+1))

counter = 0
for j in np.arange(0,len(SpiralAcquisition[0]),10): #for each point in this spiral, i.e. all of k-space
    # get mask that gets that point and it's corresponding buddy 
    mask = np.zeros(np.shape(cropped_kspace))
    
    kx = SpiralAcquisition[0][j]
    ky = SpiralAcquisition[1][j]
    mask[kx,ky]  = 1
    # now get it reflected across the origin as well 
    dkx = center - kx
    dky = center - ky
    
    mirror_kx = center + dkx 
    mirror_ky = center + dky
    mask[mirror_kx, mirror_ky] = 1
        
    # now get those points in k-space
    kspace_point = cropped_kspace*mask 
    
    #show k-space growing
    #spiraling_k = spiraling_k + kspace_point # and add it to show the growing k-space
    #ax1.imshow(np.abs(spiraling_k),vmin = 0,vmax = 6000,extent = [-.5, .5, -.5, .5],cmap = 'gray')
    #ax1.set_title('k-space used so far')
    
    # now get inverse ft of that point
    ift = np.fft.fftshift(kspace_point)
    ift = np.fft.ifft2(ift)
    iimg = np.fft.ifftshift(ift) # want to save this! 
    stacked_maskiffts[:,:,counter] = np.abs(iimg)
    counter = counter + 1
    
    
    # now begin to sum up these individual contributions
    #superimpose_img = superimpose_img + np.real(iimg)
    #ax2.imshow(np.abs(iimg),cmap = 'gray')
    #ax2.set_title('Image being produced')
    
    
    #pl.pause(.001)
    
    
    
    
    
with open('superposition_masks.pickle','wb') as handle: 
    pickle.dump(stacked_maskiffts,handle)

    
    

# a bit too computationally expensive, so gonna do it once (hopefully) and save it in a pickle file... 

In [39]:
with open('superposition.pickle','rb') as handle1:
    stack_image = pickle.load(handle1)
with open('superposition_masks.pickle','rb') as handle2:
    stack_waves = pickle.load(handle2)
with open('superposition_kspace.pickle','rb') as handle3:
    stack_kspace = pickle.load(handle3)
    
fig, (ax1, ax2, ax3,ax4) = pl.subplots(1,4)
ax4.imshow(img,cmap = 'gray')
for i in range(np.shape(stack)[2]):
    ax1.imshow(stack_kspace[:,:,i],vmin = 0,vmax = 6000,extent = [-.5, .5, -.5, .5],cmap = 'gray')
    ax2.imshow(stack_waves[:,:,i],cmap = 'gray')
    ax3.imshow(stack_image[:,:,i],cmap = 'gray')
    pl.pause(.01)

KeyboardInterrupt: 

In [28]:
with open('superposition_kspace.pickle','rb') as handle3:
    stack_kspace = pickle.load(handle3)